In [29]:
%load_ext autoreload
%autoreload 2

from data_reading import read_images, load_reference_poses, read_imu, sync_data
from visualization import visualize_trajectory
from visual_odometry import compute_relative_pose, compute_matches, add_vo_factors
from imu_preintegration import add_imu_factors

imgs = read_images()
imu = read_imu()
gt_poses = load_reference_poses()
imgs_list = []
for i, img in enumerate(imgs):
    if i>1000:
        break
    imgs_list.append(img)
synced_data = sync_data(imu, imgs_list, gt_poses)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
points0, points1, descriptors, E = compute_matches(synced_data[0].img_left_start, synced_data[0].img_right_start)
t, r = compute_relative_pose(E, points0, points1)
print(synced_data[0].gt_pose_start)
estimated_second_pose_t = synced_data[0].gt_pose_start[1].apply(t) + synced_data[0].gt_pose_start[0]
estimated_second_pose_r = synced_data[0].gt_pose_start[1] * r

pos_diff = estimated_second_pose_t - synced_data[0].gt_pose_end[0]
rot_diff = estimated_second_pose_r.as_euler('XYZ', degrees=True) - synced_data[0].gt_pose_end[1].as_euler('XYZ', degrees=True)
print(f"Position difference: {pos_diff}")
print(f"Rotation difference: {rot_diff}")


478 inliers found out of 1136 matches
(array([ 0.1762211 , -0.05578694,  0.00300311]), <scipy.spatial.transform._rotation.Rotation object at 0x7cb008892a60>)
Position difference: [-0.01363077 -0.05956058  0.99813162]
Rotation difference: [9.61597838e-05 1.10778482e-04 2.78835057e-05]


In [34]:
import numpy as np
import cv2
import gtsam
from pyLSHash import LSHash
from gtsam import DoglegOptimizer, Values, symbol_shorthand
from imu_preintegration import init_preint_params, init_imu_bias, add_gt_pose_nodes
L = symbol_shorthand.L
X = symbol_shorthand.X

def create_graph():
    #Noise on prior pose - pretty confident
    PRIOR_NOISE = gtsam.noiseModel.Diagonal.Sigmas(
        np.array([0.05, 0.05, 0.05, 0.01, 0.01, 0.01])
    )
    initial_estimates = Values()
    graph = gtsam.NonlinearFactorGraph()

    #Converting pose into Pose3 object that GTSAM can use
    pos = synced_data[0].gt_pose_start[0]
    rot = synced_data[0].gt_pose_start[1]
    R = rot.as_matrix()
    rot3 = gtsam.Rot3(R)
    point3 = gtsam.Point3(pos[0], pos[1], pos[2])
    pose3 = gtsam.Pose3(rot3, point3)
    #Using starting pose as PriorFactor
    graph.add(gtsam.PriorFactorPose3(X(0), pose3, PRIOR_NOISE))
    
    pose_count = 0
    
    keypoint_data = LSHash(
            hash_size=32,
            input_dim=256,
            num_hashtables=1,
    )
    initial_estimates.insert(X(0), gtsam.Pose3())
    add_gt_pose_nodes(graph, synced_data, initial_estimates)
    #Iterate through synced data and call factor functions on each data point
    preint_params = init_preint_params()
    preint_bias = init_imu_bias()
    add_imu_factors(graph, synced_data, initial_estimates, preint_params, preint_bias)
    # for data in synced_data:
        # try:
        #     add_vo_factors(graph, data, initial_estimates, (pose_count,pose_count+1), keypoint_data)
        #     count +=1
        # except cv2.error as e:
        #     if "five-point.cpp" in str(e) and "npoints >= 0" in str(e):
        #         print(f"Skipping frame pair {count}-{count+1} due to insufficient correspondences.")
        #         continue
        #     else:
        #         raise 
    #Run graph through Dogleg Optimizer 
    #print(initial_estimates)
    params = gtsam.DoglegParams()
    #Debugging statement
    params.setVerbosity("TERMINATION")
    optimizer = DoglegOptimizer(graph, initial_estimates, params)
    result = optimizer.optimize()
    result.print("Final results:\n")





def compare_error(estimates, real_values):
    min_length = min(len(estimates), len(real_values))
    pos_errors = []
    rot_errors = []
    for ind in range(min_length):
        real_pos, real_rot = real_values[ind].gt_pose_end
        e_pos, e_rot = estimates[ind].gt_pose_end
        pos_diff = real_pos - e_pos
        pos_error = np.linalg.norm(pos_diff)
        euler_real = real_rot.as_euler('XYZ', degrees=True)
        euler_estimate = e_rot.as_euler('XYZ', degrees=True)
        rot_error = np.linalg.norm(euler_real - euler_estimate)
        pos_errors.append(pos_error)
        rot_errors.append(rot_error)
    if min_length:
        avg_pos_error = sum(pos_errors) / len(pos_errors)
        avg_rot_error = sum(rot_errors) / len(rot_errors)
        return avg_pos_error, avg_rot_error
    return None, None




create_graph()

converged
Final results:

Values with 2997 values:
Value b0: (gtsam::imuBias::ConstantBias)
acc =  0.000142761288295 -1.96943339374e-06 -3.45985431325e-06 gyro =  7.07850895948e-07 -7.94730022077e-08 -2.47936464314e-07
errorThreshold: 1.1948407082e+15 <? 0
absoluteDecrease: 180891331 <? 1e-05
relativeDecrease: 1.51393656387e-07 <? 1e-05
iterations: 1 >? 100

Value b1: (gtsam::imuBias::ConstantBias)
acc =  0.000142761288408 -1.96943338608e-06 -3.45985434551e-06 gyro =  7.07850872522e-07 -7.94729825036e-08 -2.47936500206e-07

Value b2: (gtsam::imuBias::ConstantBias)
acc =  0.000142761288354 -1.96943338925e-06 -3.45985432846e-06 gyro =  7.07850872432e-07 -7.94729943459e-08 -2.47936489002e-07

Value b3: (gtsam::imuBias::ConstantBias)
acc =  0.000142761288356 -1.96943338938e-06  -3.4598543285e-06 gyro =  7.07850872434e-07 -7.94729943481e-08    -2.47936489e-07

Value b4: (gtsam::imuBias::ConstantBias)
acc =  0.000142761288359  -1.9694333896e-06 -3.45985432867e-06 gyro =  7.07850872434e-07   